In [1]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import threading

res_links = []
base_link = 'https://www.d-kvadrat.ru/novosti/'

for i in tqdm(range(23000, 28714)):
    res_links.append(base_link + str(i))

  0%|          | 0/5714 [00:00<?, ?it/s]

In [2]:
print(len(res_links))

5714


In [3]:
import pandas as pd
articles_links = []
df = pd.read_csv('articles_preprocessed_puncts.csv', index_col=False)
for i in range(len(df)):
    articles_links.append(df['link'][i])


In [4]:
#убери из res_links все линки, которые уже есть в articles_links

def delete_all_links(links, articles_links):
    for link in articles_links:
        links.remove(link)
    return links

In [5]:
res_links = delete_all_links(res_links, articles_links)

In [6]:
print(len(res_links))

4674


In [7]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import threading

links = []
logs = []
def check_link_is_valid(link):
    r = requests.get(link)
    if r.status_code == 200:
        # print(link)
        return True
    else:
        logs.append(link)
        return False

with tqdm(total=len(res_links)) as pbar:
    threads = []
    for i in range(len(res_links)):
        url = res_links[i]
        t = threading.Thread(target=check_link_is_valid, args=(url,))
        threads.append(t)
        # time.sleep(0.04)
        t.start()
        if len(threads) > 100:
            for t in threads:
                t.join()
            threads = []
        pbar.update(1)

for t in threads:
    t.join()

  0%|          | 0/4674 [00:00<?, ?it/s]

In [8]:
print(logs)

['https://www.d-kvadrat.ru/novosti/23028', 'https://www.d-kvadrat.ru/novosti/23171', 'https://www.d-kvadrat.ru/novosti/23226', 'https://www.d-kvadrat.ru/novosti/23233', 'https://www.d-kvadrat.ru/novosti/23219', 'https://www.d-kvadrat.ru/novosti/23235', 'https://www.d-kvadrat.ru/novosti/23247', 'https://www.d-kvadrat.ru/novosti/23256', 'https://www.d-kvadrat.ru/novosti/23334', 'https://www.d-kvadrat.ru/novosti/23283', 'https://www.d-kvadrat.ru/novosti/23297', 'https://www.d-kvadrat.ru/novosti/23324', 'https://www.d-kvadrat.ru/novosti/23279', 'https://www.d-kvadrat.ru/novosti/23307', 'https://www.d-kvadrat.ru/novosti/23309', 'https://www.d-kvadrat.ru/novosti/23308', 'https://www.d-kvadrat.ru/novosti/23366', 'https://www.d-kvadrat.ru/novosti/23416', 'https://www.d-kvadrat.ru/novosti/23609', 'https://www.d-kvadrat.ru/novosti/23615', 'https://www.d-kvadrat.ru/novosti/23636', 'https://www.d-kvadrat.ru/novosti/23624', 'https://www.d-kvadrat.ru/novosti/23640', 'https://www.d-kvadrat.ru/novosti

In [9]:
def delete_all_error_links(links, logs):
    for link in logs:
        links.remove(link)
    return links

In [10]:
links = delete_all_error_links(res_links, logs)

In [11]:
print(len(res_links))

4361


In [15]:
rus_alph = set(list('абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЫЬЭЮЯ'))

In [2]:
from tqdm.notebook import tqdm
import pandas as pd

data = []
def extract_article(link: str) -> dict:
    try:
        r = requests.get(link)
        html = BeautifulSoup(r.content, "html.parser")
        rus_alph = set(list('абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЫЬЭЮЯ'))
        title = html.find_all('h1', class_='item__title')
        title = [elem.getText() for elem in title]
        span = html.find_all('span', class_='article-page__paragraph')
        span = [elem.getText() for elem in span]
        all_paragraphs = html.find_all('p')
        only_art_text = all_paragraphs[:-2]
        clean_paragraphs = lambda s: s.replace('\n', '').replace('\t', '').replace('\r', '')
        article = [clean_paragraphs(elem.getText()) for elem in only_art_text]
        article = ' '.join(article)
        title_span_article = " ".join([span[0], article])
        drop_not_rus = lambda s: ''.join(list(filter(lambda x: x in rus_alph or x == ' ', s)))
        title_span_article = drop_not_rus(title_span_article)
        data.append({'link': link,
                     'title': title[0],
                     'article': title_span_article})

    except:
        pass

with tqdm(total=len(res_links)) as pbar:
    threads = []
    for i in range(0, len(res_links)):
        url = res_links[i]
        #max 100 threads
        if len(threads) > 100:
            for t in threads:
                t.join()
            threads = []
        t = threading.Thread(target=extract_article, args=(url,))
        threads.append(t)
        t.start()
        pbar.update(1)
        time.sleep(0.01)

for t in threads:
    t.join()

df = pd.DataFrame(data)
df.to_csv('articles_new.csv', index=False)

# with tqdm(total=10) as pbar:
#     for i in range(10):
#         url = res_links[i]
#         extract_article(url)
#         pbar.update(1)
#
# df = pd.DataFrame(data)
# df.to_csv('articles.csv', index=False)



  0%|          | 0/5714 [00:00<?, ?it/s]

In [34]:
print(len(data))

2575


In [35]:
arts_dop = pd.read_csv('articles_dop.csv', index_col=False)


In [41]:
print(arts_dop['article'][5])

На трассе Ижевск  Сарапул установили камеру фиксирующую разговоры по телефону за рулем Камера которая регистрирует разговоры по телефону без гарнитуры и непристегнутый ремень на дороге Ижевск  Сарапул  единственная в Удмуртии Об этом сообщает прессслужба правительства УР С помощью нейросети камера фиксирует новые виды нарушений ПДД В модель загрузили множество фотографий пристегнутых и непристёгнутых ремнем безопасности водителей а также тех кто управляет автомобилем с телефоном в руке   Таким образом нейосеть обучили выбирать изображения с нарушениями У камеры  высокое разрешение что позволяет прибору  улавливать незначительные детали Выбранные нейросетью снимки передаются в ГИБДД По словам министра транспорта Удмуртии Алексея Горбачева такая камера крайне необходима для безопасности на дорогах  Установлено что почти половина российских водителей отвлекаются на гаджеты при вождении Даже один взгляд в телефон затормаживает реакцию и может стать причиной ДТП  объяснил он необходимость у

In [12]:
# все линки помести в массив

links_art = links
print(len(links_art))
print(links_art[0])


4361
https://www.d-kvadrat.ru/novosti/23000


In [13]:
from tqdm.notebook import tqdm
import pandas as pd

data = []
def extract_article(link: str) -> dict:
    try:
        r = requests.get(link)
        html = BeautifulSoup(r.content, "html.parser")
        # нужно забрать текста статьи и спан, но нужно оставить русские буквы и пунктуацию
        punct = set(list('.,!?;:'))
        rus_alph = set(list('абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЫЬЭЮЯ'))
        title = html.find_all('h1', class_='item__title')
        title = [elem.getText() for elem in title]
        span = html.find_all('span', class_='article-page__paragraph')
        span = [elem.getText() for elem in span]
        all_paragraphs = html.find_all('p')
        only_art_text = all_paragraphs[:-2]
        clean_paragraphs = lambda s: s.replace('\n', '').replace('\t', '').replace('\r', '')
        article = [clean_paragraphs(elem.getText()) for elem in only_art_text]
        article = ' '.join(article)
        title_span_article = " ".join([span[0], article])
        drop_not_rus = lambda s: ''.join(list(filter(lambda x: x in rus_alph or x in punct or x == ' ', s)))
        title_span_article = drop_not_rus(title_span_article)
        data.append({'link': link,
                     'title': title[0],
                     'article': title_span_article})



    except:
        pass

with tqdm(total=len(links_art)) as pbar:
    threads = []
    for i in range(0, len(links_art)):
        url = links_art[i]
        if len(threads) > 300:
            for t in threads:
                t.join()
            threads = []
        t = threading.Thread(target=extract_article, args=(url,))
        threads.append(t)
        t.start()
        pbar.update(1)
        time.sleep(0.01)

for t in threads:
    t.join()

df = pd.DataFrame(data)



  0%|          | 0/4361 [00:00<?, ?it/s]

In [14]:
df

,link,title,article
0,https://www.d-kvadrat.ru/novosti/23002,В «Ижавиа» рассказали о правилах провоза лыжно...,В Ижавиа рассказали о правилах провоза спортив...
1,https://www.d-kvadrat.ru/novosti/23004,Сегодня индекс МосБиржи продолжит проторговыва...,Сегодня индекс МосБиржи продолжит проторговыва...
2,https://www.d-kvadrat.ru/novosti/23000,Алан Лушников: калибр армейских автоматов нужн...,Президент концерна Калашников Алан Лушников за...
3,https://www.d-kvadrat.ru/novosti/23007,Bloomberg: рецессия в России продлится до III ...,Российская экономика начнет расти только в тре...
4,https://www.d-kvadrat.ru/novosti/23008,Подведены предварительные итоги здравоохранени...,О предварительных итогах работы отрасли здраво...
...,...,...,...
4356,https://www.d-kvadrat.ru/novosti/28709,Удмуртия вошла в ТОП-5 «зеленых» регионов России,Удмуртия продемонстрировала высокий уровень ра...
4357,https://www.d-kvadrat.ru/novosti/28712,С приближением заседания ЦБ инверсия кривой ОФ...,С приближением заседания ЦБ инверсия кривой ОФ...
4358,https://www.d-kvadrat.ru/novosti/28708,100 школьников Ижевска посетили музей ИРЗ в ра...,В ноябре более школьников классов посетили...
4359,https://www.d-kvadrat.ru/novosti/28711,Сотрудник ИРЗ на доске почета УР,К Дню государственности Удмуртии была обновлен...


In [15]:
df.to_csv('sampling_arts.csv', index=False)

In [28]:
#в df_art добавь отдельной колонкой статьи из df в соответствии по link

def add_article(df_art, df):
    df_art['article_punct'] = ''
    for i in range(len(df_art)):
        for j in range(len(df)):
            if df_art['link'][i] == df['link'][j]:
                df_art['article_punct'][i] = df['article'][j]
    return df_art




In [29]:
df_art = add_article(df_art, df)

/var/folders/rn/rmhpxg1178d0qhdyy8l2xg200000gp/T/ipykernel_1678/4241704524.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_art['article_punct'][i] = df['article'][j]
/var/folders/rn/rmhpxg1178d0qhdyy8l2xg200000gp/T/ipykernel_1678/4241704524.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_art['article_punct'][i] = df['article'][j]
/var/folders/rn/rmhpxg1178d0qhdyy8l2xg200000gp/T/ipykernel_1678/4241704524.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [31]:
df_art.to_csv('articles_preprocessed_puncts.csv', index=False)